# Looping through tip racks

In the `pylabrobot.resources.functional` module, we have utilities for looping through tip spots in one or more tip racks. They support caching to disk, so that you can resume where you left off if your script is interrupted.

In [1]:
# instantiate some hamilton tip racks as an example
from pylabrobot.resources.hamilton import HT # an example tip rack
tip_rack_0 = HT(name='tip_rack_0')
tip_rack_1 = HT(name='tip_rack_1')
tip_racks = [tip_rack_0, tip_rack_1]

Tip spot generators take a list of tip spots (`list[TipSpot]`) as an argument. With `F.get_all_tip_spots`, you can get all tip spots in one or more tip racks. The tip spots will be column-first, i.e. the first tip spot is the top left corner, the second tip spot is the one below it, and so on.

In [2]:
import pylabrobot.resources.functional as F
tip_spots = F.get_all_tip_spots(tip_racks)
tip_spots[0]

TipSpot(name=tip_rack_0_tipspot_0_0, location=Coordinate(007.200, 068.300, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)

## Basic linear generator 

The linear generator will loop through all tip spots in the order they are given, with the option to repeat.

In [3]:
linear_generator = F.linear_tip_spot_generator(
  tip_spots=tip_spots,                      # the list of tip spots to use
  cache_file_path="./linear_cache.json",    # load/save tip spot cache for state in between runs
  repeat=False,                             # don't repeat the tip spots if they run out
)

Tip spot generators are asynchronous, so use `await` and `__anext__` to get the next tip spot.

In [4]:
await linear_generator.__anext__()

TipSpot(name=tip_rack_0_tipspot_0_0, location=Coordinate(007.200, 068.300, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)

To get multiple tip spots, call `__anext__` multiple times.

In [5]:
N = 3
tip_spots = [await linear_generator.__anext__() for _ in range(N)]
[ts.name for ts in tip_spots]

['tip_rack_0_tipspot_0_1', 'tip_rack_0_tipspot_0_2', 'tip_rack_0_tipspot_0_3']

Save the state of the generator at an arbitrary point by calling `save_state`. This method will be called automatically when the program crashes or is interrupted.

In [6]:
linear_generator.save_state()

Override the index by calling `set_index`.

In [7]:
linear_generator.set_index(12)
await linear_generator.__anext__()

TipSpot(name=tip_rack_0_tipspot_1_4, location=Coordinate(016.200, 032.300, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)

## Randomized generator

The randomized generator will loop through all tip spots in a random order, with the option to repeat. If repeating, set the parameter `K` to not sample a tip spot that has been sampled in the last `K` samples.

In [8]:
random_generator = F.randomized_tip_spot_generator(
  tip_spots=tip_spots,                      # the list of tip spots to use
  cache_file_path="./random_cache.json",    # load/save tip spot cache for state in between runs
  K=10,                                     # don't sample tip spots that have been used in the last K samples
)

In [9]:
await random_generator.__anext__()

TipSpot(name=tip_rack_0_tipspot_0_3, location=Coordinate(007.200, 041.300, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)